In [ ]:
from __future__ import print_function
import os,sys
import ROOT as rt
from larcv import larcv
from larlite import larlite
import chart_studio.plotly as py
import plotly.graph_objects as go
import numpy as np

In [ ]:
import lardly
detdata = lardly.DetectorOutline()

In [ ]:
# DATA FILES

datafolder="testdata/mcc9_v29e_dl_run3b_bnb_nu_overlay_nocrtremerge/"
dlmergedfile=datafolder+"/merged_dlreco_3ab3a878-3aea-4bc7-926e-bbb7554875f1.root"
larmatchfile=datafolder+"/larmatchme_dlgen2.1_merged_dlreco_3ab3a878-3aea-4bc7-926e-bbb7554875f1_larlite.root"
kpsrecoana="test_recov2_kpsrecomanagerana.root"
kpsrecolarlite="test_recov2_larlite.root"

# Gen-2 reco products
kpsrecofile = rt.TFile(kpsrecoana)
kpsreco = kpsrecofile.Get("KPSRecoManagerTree")

# LArCV data
iolcv = larcv.IOManager(larcv.IOManager.kREAD,"larcv",larcv.IOManager.kTickBackward)
iolcv.add_in_file( dlmergedfile)
iolcv.specify_data_read( larcv.kProductImage2D, "wire" );
iolcv.specify_data_read( larcv.kProductChStatus, "wire" );
iolcv.specify_data_read( larcv.kProductImage2D, "ubspurn_plane0" )
iolcv.specify_data_read( larcv.kProductImage2D, "ubspurn_plane1" )
iolcv.specify_data_read( larcv.kProductImage2D, "ubspurn_plane2" )
iolcv.specify_data_read( larcv.kProductSparseImage, "sparseuresnetout" )                                                                                                                  
iolcv.reverse_all_products()
iolcv.initialize()

# upstream larlite data
ioll = larlite.storage_manager(larlite.storage_manager.kREAD)
ioll.set_verbosity(1)
ioll.add_in_filename( dlmergedfile )
ioll.add_in_filename( larmatchfile )
ioll.set_data_to_read( larlite.data.kLArFlow3DHit, "larmatch" )
ioll.set_data_to_read( larlite.data.kTrack,  "trackReco" )
ioll.set_data_to_read( larlite.data.kMCShower, "mcreco" )
ioll.set_data_to_read( larlite.data.kMCTrack,  "mcreco" )
ioll.set_data_to_read( larlite.data.kMCTruth,  "generator" )
ioll.set_data_to_read( larlite.data.kOpFlash,  "simpleFlashBeam" )
ioll.set_data_to_read( larlite.data.kOpFlash,  "simpleFlashCosmic" )
ioll.open()

# kps larlite output
kpsll = larlite.storage_manager(larlite.storage_manager.kREAD)
kpsll.add_in_filename( kpsrecolarlite )
kpsll.set_verbosity(1)
kpsll.open()


In [ ]:
# SET THE ENTRY
INPUT_ENTRY = 1
KPSOUT_ENTRY = 1
iolcv.read_entry(INPUT_ENTRY)
ioll.go_to(INPUT_ENTRY)
kpsll.go_to(KPSOUT_ENTRY)
kpsreco.GetEntry(KPSOUT_ENTRY)

In [ ]:
print(ioll.run_id()," ",ioll.subrun_id()," ",ioll.event_id())

In [ ]:
# Add LARMATCH visualization
ev_larmatch = ioll.get_data(larlite.data.kLArFlow3DHit,"larmatch")
print("LM Hits: ",ev_larmatch.size())
lm_plot = lardly.data.visualize_larlite_larflowhits( ev_larmatch, name="larmatch" )

In [ ]:
# ADD (IN-TIME) RECO KEYPOINTS

KEYPOINT_COLORS={0:"rgb(255,0,0)",   # nu
                 1:"rgb(255,0,255)", # track-start
                 2:"rgb(255,0,127)", # track-end
                 3:"rgb(255,127,0)", # shower start
                 4:"rgb(0,255,255)", # michel
                 5:"rgb(255,255,0)", # delta
                 6:"rgb(50,50,50)"   # other?
                }

ev_reco_keypoints = kpsll.get_data(larlite.data.kLArFlow3DHit,"keypoint")
kp_plots = {0:[],1:[],2:[],3:[],4:[],5:[],6:[]}

for ikp in range(ev_reco_keypoints.size()):
    kp = ev_reco_keypoints.at(ikp)
    kptype = int(kp[3])
    kpscore = float(kp[4])
    kptrace = {
        "type":"scatter3d",
        "x": [kp[0]],
        "y": [kp[1]],
        "z": [kp[2]],
        "mode":"markers",
        "name":"[%d] %.1f"%(kptype,kpscore),
        "marker":{"color":KEYPOINT_COLORS[kptype],"size":5,"opacity":0.9},
    }
    kp_plots[kptype].append(kptrace)


In [ ]:
# ADD (OUT-OF-TIME) RECO KEYPOINTS

COSMIC_KEYPOINT_COLORS={1:"rgb(0,125,255)", # track-start
                        2:"rgb(0,0,255)" # track-end
                        }

ev_reco_keypoints = kpsll.get_data(larlite.data.kLArFlow3DHit,"keypointcosmic")
kp_cosmic_plots = {0:[],1:[],2:[],3:[],4:[],5:[]}

for ikp in range(ev_reco_keypoints.size()):
    kp = ev_reco_keypoints.at(ikp)
    kptype = int(kp[3])
    kpscore = float(kp[4])
    kptrace = {
        "type":"scatter3d",
        "x": [kp[0]],
        "y": [kp[1]],
        "z": [kp[2]],
        "mode":"markers",
        "name":"[%d] %.1f"%(kptype,kpscore),
        "marker":{"color":COSMIC_KEYPOINT_COLORS[kptype],"size":5,"opacity":0.9},
    }
    kp_cosmic_plots[kptype].append(kptrace)

In [ ]:
# get the traces for the nu interaction truth
nutruth = lardly.data.visualize_nu_interaction( ioll, do_sce_correction=True  )
#print(nutruth)

In [ ]:
# Find closest Gen-2 Vertex
min_dist = 1e9
min_index = 0
nvertices = kpsreco.nufitted_v.size()
print("Num vertices",nvertices)

for i in range( kpsreco.nufitted_v.size() ):
    nuvtx = kpsreco.nufitted_v.at(i)
    dist = 0.0
    for v in range(3):
        dist += (nuvtx.pos[v]-gen1_vtx[v])*(nuvtx.pos[v]-gen1_vtx[v])
    if min_dist>dist:
        min_dist = dist
        min_index = i
        
gen2_nuvtx = kpsreco.nufitted_v.at(min_index)
print("Closest Gen-2 Vertex: idx=%d dist=%.2f cm"%(min_index,min_dist))
print("  TYPE: ",gen2_nuvtx.keypoint_type)

# Make track and shower 3d scatter plots
ntracks  = gen2_nuvtx.track_v.size()
nshowers = gen2_nuvtx.shower_v.size()
print("Num tracks=",ntracks)
print("Num showers=",nshowers)

# PLOT TRACK FOR VERTEX                                                                                                                                               
for itrack in range(ntracks):
    # track-cluster
    
    lfcluster = gen2_nuvtx.track_hitcluster_v[itrack]
    cluster_trace = lardly.data.visualize_larlite_larflowhits( lfcluster, name="c[%d]"%(itrack) )
    cluster_trace["marker"]["color"] = "rgb(0,255,0)"
    cluster_trace["marker"]["opacity"] = 0.8
    cluster_trace["marker"]["size"] = 2.0
    plotdata.append(cluster_trace)


In [ ]:
# Find Nu Gen-2 Vertex
nvertices = kpsreco.nufitted_v.size()
print("Num vertices",nvertices)


nu_score = 0.0
nu_vtx_index = -1
for i in range( kpsreco.nufitted_v.size() ):
    nuvtx = kpsreco.nufitted_v.at(i)
    

gen2_nuvtx = kpsreco.nufitted_v.at(min_index)
print("Closest Gen-2 Vertex: idx=%d dist=%.2f cm"%(min_index,min_dist))
print("  TYPE: ",gen2_nuvtx.keypoint_type)

# Make track and shower 3d scatter plots
ntracks  = gen2_nuvtx.track_v.size()
nshowers = gen2_nuvtx.shower_v.size()
print("Num tracks=",ntracks)
print("Num showers=",nshowers)

# PLOT TRACK FOR VERTEX                                                                                                                                               
for itrack in range(ntracks):
    # track-cluster
    print("Make Plots to Track[%d]"%(itrack))
    lfcluster = gen2_nuvtx.track_hitcluster_v[itrack]
    cluster_trace = lardly.data.visualize_larlite_larflowhits( lfcluster, name="c[%d]"%(itrack) )
    cluster_trace["marker"]["color"] = "rgb(0,255,0)"
    cluster_trace["marker"]["opacity"] = 0.8
    cluster_trace["marker"]["size"] = 2.0
    plotdata.append(cluster_trace)
    
    # track-line                                                                                                                                                      
    track = gen2_nuvtx.track_v.at(itrack)
    trktrace = lardly.data.visualize_larlite_track( track )
    trktrace["name"] = "T[%d]"%(itrack)
    trktrace["line"]["color"] = "rgb(0,255,255)"
    trktrace["line"]["width"] = 5
    trktrace["line"]["opacity"] = 1.0
    #plotdata.append( trktrace )

    kemu = gen2_nuvtx.track_kemu_v.at(itrack)
    kep  = gen2_nuvtx.track_keproton_v.at(itrack)
    llmu = gen2_nuvtx.track_muid_v.at(itrack)
    llp  = gen2_nuvtx.track_protonid_v.at(itrack)
    llr  = gen2_nuvtx.track_mu_vs_proton_llratio_v.at(itrack)
    print("  TRACK[%d] KE(mu)=%.2f KE(p)=%.2f -log(L)_mu=%.2f -log(L)_p=%.2f LLratio=%.2f"%(itrack,kemu,kep,llmu,llp,llr))

# PLOT SHOWER FOR VERTEX                                                                                                                                              
for ishower in range(gen2_nuvtx.shower_v.size()):
    shower = gen2_nuvtx.shower_v.at(ishower)
    shower_trunk = gen2_nuvtx.shower_trunk_v.at(ishower)
    shower_pca   = gen2_nuvtx.shower_pcaxis_v.at(ishower)

    cluster_trace = lardly.data.visualize_larlite_larflowhits( shower, name="S[%d] N[%d]"%(ishower,shower.size()) )
    trunk_trace   = lardly.data.visualize_larlite_track( shower_trunk )

    rgbcolor = np.random.randint(255,size=3)
    cluster_trace["marker"]["color"] = "rgb(%d,%d,0)"%(rgbcolor[0],rgbcolor[1])
    #cluster_trace["marker"]["opacity"] = 0.8
    cluster_trace["marker"]["size"] = 2.0
    plotdata.append(cluster_trace)

    #trunk_trace["line"]["color"] = "rgb(200,0,200)"
    #trunk_trace["line"]["width"] = 5
    #trunk_trace["line"]["opacity"] = 1.0
    #trunk_trace["name"] = "TRNK[%d]"%(ishower)
    #plotdata.append(trunk_trace)

    #pcatrace = lardly.data.visualize_pcaxis( shower_pca )
    #pcatrace["name"] = "SHR[%d]"%(ishower)
    #pcatrace["line"]["color"] = "rgb(255,0,0)"
    #pcatrace["line"]["width"] = 5
    #pcatrace["line"]["opacity"] = 1.0
    #plotdata.append( pcatrace )

    shower_mom_v = gen2_nuvtx.shower_plane_mom_vv.at(ishower)
    sh_p0 = shower_mom_v[0].E()
    sh_p1 = shower_mom_v[1].E()
    sh_p2 = shower_mom_v[2].E()

    shower_dqdx_v = gen2_nuvtx.shower_plane_dqdx_vv.at(ishower)
    dqdx_p0 = shower_dqdx_v[0]
    dqdx_p1 = shower_dqdx_v[1]
    dqdx_p2 = shower_dqdx_v[2]
    print("  SHOWER[%d] KE=(%.2f,%.2f,%.2f) dq/dx=(%.2f,%.2f,%.2f)"%(ishower,sh_p0,sh_p1,sh_p2,dqdx_p0,dqdx_p1,dqdx_p2))





In [ ]:
# MAKE OPFLASH OBJECTS

ev_flash = ioll.get_data(larlite.data.kOpFlash,"simpleFlashBeam")
nflashes = 0
flash_plots = []
for iflash in range(ev_flash.size()):
    flash = ev_flash.at(iflash)
    if flash.Time()>2.94 and flash.Time()<4.86:
        flash_trace_v = lardly.data.visualize_larlite_opflash_3d( flash )
        flash_plots += flash_trace_v
        nflashes += 1
        break
if nflashes==0:
    flash_plots += lardly.data.visualize_empty_opflash()


In [ ]:
# FORM THE PLOT: SHOWING KEYPOINT OUTPUT

# DATA
data = [] + detdata.getlines(color=(10,10,10))
#data += [lm_plot]
#for kptype in range(6,-1,-1):
#    data += kp_plots[kptype]
#data += kp_cosmic_plots[1]
#data += kp_cosmic_plots[2]
#data += nutruth

# LAYOUT
axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(100, 100, 100,0.5)",
    "gridcolor": "rgb(50, 50, 50)",
    "zerolinecolor": "rgb(0, 0, 0)",
}


layout = go.Layout(
    title='Neutrino Interaction',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 3},
        "camera": {"eye": {"x": -2, "y": 1, "z": 1},
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=data, layout=layout)
fig.show()

In [ ]:
# U-PLANE
img_u = iolcv.get_data(larcv.kProductImage2D,"wire").as_vector().at(0) 
plot_uplane = [ lardly.data.visualize_larcv_image2d( img_u ) ]

fig = go.Figure(data=plot_uplane)
fig.show()

In [ ]:
# V-PLANE
img_v = iolcv.get_data(larcv.kProductImage2D,"wire").as_vector().at(1) 
plot_vplane = [ lardly.data.visualize_larcv_image2d( img_v ) ]

fig = go.Figure(data=plot_vplane)
fig.show()

In [ ]:
# Y-PLANE
img_y = iolcv.get_data(larcv.kProductImage2D,"wire").as_vector().at(2) 
plot_yplane = [ lardly.data.visualize_larcv_image2d( img_y ) ]

fig = go.Figure(data=plot_yplane)
fig.show()

In [ ]:
# Analysis Variables
print "Enu: %.2f MeV"%(fvv.Enu_1e1p)
print "BDT: %.4f"%(fvv.BDTscore_1e1p)
print "showerQ=(%.2f,%.2f,%.2f)"%(fvv.shower1_sumQ_U, fvv.shower1_sumQ_V, fvv.shower1_sumQ_Y)